In [3]:
# Pandas and numpy necessary to do basic data cleaning
import pandas as pd
import numpy as np

# Import and Clean Data

In [4]:
# Import
df = pd.read_stata("./aer_data/aer_data.dta")

# Clean up data types
df['year'] = df.year.dt.year
for x in ['year','fips','stfips']:
    df[x] = df[x].astype(int)

# Filter data the same as paper authors
df = df[df.year <= 1988]
df = df[~((df.stfips == 36) & (df.cofips == 61))] 
df = df[~((df.stfips == 6) & (df.cofips == 37))]
df = df[~((df.stfips == 17) & (df.cofips == 31))]

# Make new variables related to urbanization
df_1960_pcturban = df.loc[df.year == 1960, ['fips','D_60pcturban_t']] \
                     .drop_duplicates('fips') \
                     .rename(columns={'D_60pcturban_t':'_urb'}) \
                     .fillna(0)
df = df.merge(df_1960_pcturban, how='left', on='fips')
df['Durb'] = pd.cut(x=df._urb, bins=[0,1,25,50,75,110], right=False, labels=[0,1,25,50,75])

# Make straight-up dummy variables
year_dummies   = pd.get_dummies(df.year, prefix="_Iyear", drop_first=True)
Durb_dummies   = pd.get_dummies(df.Durb, prefix="_IDurb", drop_first=True)
fips_dummies   = pd.get_dummies(df.fips, prefix="_Ifips", drop_first=True)
stfips_dummies = pd.get_dummies(df.stfips, prefix="_Istfips", drop_first=True)

# Make interaction dummies
for year in np.sort(df.year.unique()):
    for Durb in np.sort(df.Durb.unique()):
        if (f"_Iyear_{year}" in year_dummies.columns) & (f"_IDurb_{Durb}" in Durb_dummies.columns):
            df[f"_IyeaXDur_{year}_{Durb}"] = year_dummies[f"_Iyear_{year}"]*Durb_dummies[f"_IDurb_{Durb}"]
    for stfips in np.sort(df.stfips.unique()):
        if (f"_Iyear_{year}" in year_dummies.columns) & (f"_Istfips_{stfips}" in stfips_dummies.columns):
            df[f"_IyeaXstf_{year}_{stfips}"] = year_dummies[f"_Iyear_{year}"]*stfips_dummies[f"_Istfips_{stfips}"]
for fips in np.sort(df.fips.unique()):
    df[f"_IfipXyea_{fips}"] = np.where((df['fips']) == fips, df['year'], 0)

# Make did1 dummies
for i,did1 in enumerate(np.sort(df.did1.unique())):
    if did1 != -1:
        df[f"_DDdid1_{i+1}"] = np.where(df['did1'] == did1, 1, 0)

# Add on year and fips dummies
df = pd.concat([df,year_dummies],axis=1)
df = pd.concat([df,fips_dummies],axis=1)

# Drop missing response values
df = df[~df['amr'].isna()]

# Add constant
df['_const']  = 1

FileNotFoundError: [Errno 2] No such file or directory: './aer_data/aer_data.dta'

In [5]:
def get_XY_arrays(df,X_cols,Y_var,weight_var):

    # Make arrays 
    Y      = df[Y_var].to_numpy()[:,None]
    X      = df[X_cols].to_numpy()
    weight = df[weight_var].to_numpy()[:,None]

    return Y,X,weight

In [2]:
# Define covariates to use
_Ifips_cols   = [col for col in df if col.startswith('_Ifips')]
_Iyear_cols   = [col for col in df if col.startswith('_Iyear')]
_IyeaXDu_cols = [col for col in df if col.startswith('_IyeaXDu')]
_DD_cols      = [col for col in df if col.startswith('_DD')]
X_cols        = _Ifips_cols+_Iyear_cols+_IyeaXDu_cols+_DD_cols+['_const'] 
Y_var         = 'amr'
weight_var    = 'popwt'

Y,X,weight = make_regression_matrices(df,X_cols,Y_var,weight_var)

NameError: name 'df' is not defined

## Get Coefficient and SE Estimates

In [6]:
def run_fe_weighted_regression(y,X,w_popwt,df):
    
    # Get beta value
    beta = np.linalg.pinv(np.multiply(X,w_popwt).T @ X) @ np.multiply(X,w_popwt).T @ y

    # Build the 'meat' of the cluster sandwich SE estimator
    clust_cov_sum = np.zeros((len(beta),len(beta)))
    for clust in np.sort(df.fips.unique()):

        # Define data just from cluster
        df_clust = df[df.fips == clust]
        X_clust  = df_clust[X_cols].to_numpy()
        y_clust  = df_clust['amr'].to_numpy()[:,None]
        w_popwt_clust = df_clust['popwt'].to_numpy()[:,None]

        # Do weighted cluster robust SE formula
        u_j  = np.multiply((y_clust - X_clust @ beta), X_clust)
        wu_j = np.multiply(w_popwt_clust, u_j)
        clust_sum = np.sum(wu_j, axis=0)[None,:]
        clust_cov = clust_sum.T @ clust_sum

        # Add to overall 
        clust_cov_sum += clust_cov

    # Get (X'X)^(-1): the 'bread' of the sandwich
    vcov = np.linalg.pinv(np.multiply(X,w_popwt).T @ X)
    vcov = np.where(vcov < 0, 0, vcov)

    # Finite-sample correction
    n_clust = df.fips.unique().shape[0]
    N       = np.sum(df['popwt'])
    k       = beta.shape[0]
    qc      = (n_clust/(n_clust-1)) * (N/(N-k))

    # Get standard errors of betas
    SE = np.sqrt(np.diag(qc * vcov @ clust_cov_sum @ vcov))
    
    return beta,vcov,SE

In [7]:
beta,vcov,SE = run_fe_weighted_regression(y,X,w_popwt,df)

## Show Results in Table and Export

In [8]:
beta_table = pd.DataFrame.from_dict({"beta_name":X_cols,
                                     "beta_values":beta.ravel(),
                                     "beta_se":SE})
beta_table = beta_table[~beta_table.beta_name.str.contains('^_Ifips_')]
print(beta_table.head(15))
print(beta_table.tail(15))

        beta_name  beta_values    beta_se
3061  _Iyear_1960    12.672612  26.253466
3062  _Iyear_1961   -15.191984  25.961281
3063  _Iyear_1962     7.382851  27.877679
3064  _Iyear_1963    25.784916  31.001602
3065  _Iyear_1964     2.305644  38.843589
3066  _Iyear_1965     0.061686  35.270078
3067  _Iyear_1966    -0.821723  36.798754
3068  _Iyear_1967   -16.789440  37.756279
3069  _Iyear_1968     1.279656  41.673243
3070  _Iyear_1969    -2.463052  42.303299
3071  _Iyear_1970   -26.824601  40.544035
3072  _Iyear_1971   -50.165184  43.566103
3073  _Iyear_1972   -47.168861  44.770084
3074  _Iyear_1973   -56.113462  48.548970
3075  _Iyear_1974  -105.138011  45.283146
              beta_name  beta_values    beta_se
3198   _IyeaXDur_1987_1   -12.613402  57.973286
3199  _IyeaXDur_1987_25   -11.973542  57.578462
3200  _IyeaXDur_1987_50   -16.232936  58.899123
3201  _IyeaXDur_1987_75   -18.219819  62.532406
3202   _IyeaXDur_1988_1   -12.366518  59.497630
3203  _IyeaXDur_1988_25   -14.603190  59

In [9]:
# Export
beta_table = beta_table[beta_table.beta_name.str.contains('^_DDdid1_')]
beta_table.to_csv('table2_replication.csv')